In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

2023-11-28 21:32:59.655114: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
tf.__version__

'2.4.0'

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[1], 'GPU')

2023-11-28 21:33:00.178222: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-28 21:33:00.178631: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-11-28 21:33:00.180997: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 21:33:00.181070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 4060 Ti computeCapability: 8.9
coreClock: 2.655GHz coreCount: 34 deviceMemorySize: 15.71GiB deviceMemoryBandwidth: 268.25GiB/s
2023-11-28 21:33:00.181086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

# Create a Submission File 

The submitted file must include all the predictions from Dan, Kyu, and Play Style testing CSVs. Therefore, you must train your models before submitting any file. If your model is not trained for Dan, Kyu, or Play Style, check out the previous tutorials about how to train your model or just simply use the baseline models (model_dan_tutorial.h5, model_kyu_tutorial.h5, model_playstyle.h5) 

**PublicUpload.csv** must be in the following form:
```
DTPU0000000001,id,qr,pq,pd,ab
DTPU0000000002,ao,ab,ha,ff,qd
DTPU0000000003,qd,gd,fh,ed,fa
DTPU0000000004,pr,ba,dq,hh,aj
DTPU0000000005,ph,jh,af,df,gj
.
.
.
KTPU0000000001,id,qr,pq,pd,ab
KTPU0000000002,ao,ab,ha,ff,qd
KTPU0000000003,qd,gd,fh,ed,fa
KTPU0000000004,pr,ba,dq,hh,aj
KTPU0000000005,ph,jh,af,df,gj
.
.
.
PSTPU0000000001_79,2
PSTPU0000000001_55,3
PSTPU0000000001_105,2
PSTPU0000000002_37,1
PSTPU0000000002_113,3
```

**For Dan and Kyu:**
- Column 1: Game ID
- Column 2: Predicted Moves, up to 5 predictions for each game

**For PlayStyle**:
- Column 1: Game ID
- Column 2: Predicted Game Style

The code block below is copied from the tutorial notebooks and it includes the pre-processing methods of the trained models. You might need to change the functions below with respect to your models.

In [4]:
chars = 'abcdefghijklmnopqrs'
coordinates = {k:v for v,k in enumerate(chars)}
chartonumbers = {k:v for k,v in enumerate(chars)}

def prepare_input_for_dan_kyu_models(moves):
    x = np.zeros((19,19,10))
    map = []
    # x[:,:,6] = 1
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        map.append(row)
        map.append(column)
        if color == 'B':
            x[row,column,0] = 1
            x[row,column,2] = 1
            # x[:,:,6] = 0
        if color == 'W':
            x[row,column,1] = 1
            x[row,column,2] = 1
            # x[:,:,6] = 0
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,3] = 1

        color = moves[-1][0]
        if color == 'B':
            x[last_move_row,last_move_column,4] = 1
        else:
            x[last_move_row,last_move_column,5] = 1

        # #最後一子的周邊分布
        # for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:   
        #     if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
        #         x[dr,dc,6]=1
        # #氣
        # for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:
        #     if color == 'B':
        #         if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,5]:
        #             x[dr,dc,7]=1 
        #     else:
        #         if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,4]:
        #             x[dr,dc,8]=1

    for i in range(0, len(map), 2):
        qi = 4
        for dr, dc in [(map[i], map[i+1]+1), (map[i], map[i+1]-1), (map[i]+1, map[i+1]), (map[i]-1, map[i+1])]:   
            if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
                qi-=1
        if qi==1:
            x[map[i],map[i+1],6]=1
        if qi==2:
            x[map[i],map[i+1],7]=1
        if qi==3:
            x[map[i],map[i+1],8]=1
        if qi==4:
            x[map[i],map[i+1],9]=1


    x[:,:,2] = np.where(x[:,:,2] == 0, 1, 0)
    return x



def prepare_input_for_playstyle_model(moves):
    x = np.zeros((19,19,15))
    map=[]
    for move in moves:
        color = move[0]
        column = coordinates[move[2]]
        row = coordinates[move[3]]
        map.append(row)
        map.append(column)
        x[row,column,0] = 1
        if color == 'B':
            x[row,column,4] = 1
        else:
            x[row,column,5] = 1
    if moves:
        last_move_column = coordinates[moves[-1][2]]
        last_move_row = coordinates[moves[-1][3]]
        x[row,column,1] = 1

        #最後一子的顏色
        color = moves[-1][0]
        if color == 'B':
            x[last_move_row,last_move_column,2] = 1
        else:
            x[last_move_row,last_move_column,3] = 1
        
        #最後一子的周邊分布
        for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:   
            if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
                x[dr,dc,6]=1

        #倒數第二子的氣
        second_last_move_column = coordinates[moves[-1][2]]
        second_last_move_row = coordinates[moves[-1][3]]
        for dr, dc in [(second_last_move_row, second_last_move_column+1), (second_last_move_row, second_last_move_column-1), (second_last_move_row+1, second_last_move_column), (second_last_move_row-1, second_last_move_column),(second_last_move_row-1, second_last_move_column+1),(second_last_move_row-1, second_last_move_column-1),(second_last_move_row+1, second_last_move_column-1),(second_last_move_row+1, second_last_move_column+1)]:   
            if color == 'B':
                if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,5]:
                    x[dr,dc,7]=1 
            else:
                if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,4]:
                    x[dr,dc,8]=1 

        #氣
        for dr, dc in [(last_move_row, last_move_column+1), (last_move_row, last_move_column-1), (last_move_row+1, last_move_column), (last_move_row-1, last_move_column)]:
            if color == 'B':
                if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,5]:
                    x[dr,dc,9]=1 
            else:
                if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,4]:
                    x[dr,dc,10]=1

             
    for i in range(0, len(map), 2):
        # if (map[i+1] == 0 and map[i]== 0) or (map[i+1] == 18 and map[i]== 18) or (map[i+1] == 0 and map[i]== 18) or (map[i+1] == 18 and map[i]== 0):
        #     qi = 2
        # elif map[i+1] == 0 or map[i+1] == 18 or map[i] == 0 or map[i] == 18:
        #     qi = 3
        # else:
        qi = 4

        for dr, dc in [(map[i], map[i+1]+1), (map[i], map[i+1]-1), (map[i]+1, map[i+1]), (map[i]-1, map[i+1])]:   
            if 0 <= dr < 19 and 0<= dc < 19 and not x[dr,dc,0]:
                qi-=1
        if qi==1:
            x[map[i],map[i+1],11]=1
        if qi==2:
            x[map[i],map[i+1],12]=1
        if qi==3:
            x[map[i],map[i+1],13]=1
        if qi==4:
            x[map[i],map[i+1],14]=1
    
    return x

def number_to_char(number):
    number_1, number_2 = divmod(number, 19)
    return chartonumbers[number_1] + chartonumbers[number_2]

def top_5_preds_with_chars(predictions):
    resulting_preds_numbers = [np.flip(np.argpartition(prediction, -5)[-5:]) for prediction in predictions]
    resulting_preds_chars = np.vectorize(number_to_char)(resulting_preds_numbers)
    return resulting_preds_chars

## Save Predictions for Dan
The code block below is to use **dan model** to predict and save the results in required form. It generates the best 5 predictions for each sample and convert them to character coordinates.

In [5]:
# Load your own model. Here we use the baseline model
model = load_model('./model_bestdan.h5')

# Load the corresponding dataset
df = open('./29_Public Testing Dataset_v2/dan_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

2023-11-28 21:33:00.218892: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-11-28 21:33:00.218970: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-11-28 21:33:00.219050: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:05:00.0 name: NVIDIA GeForce RTX 4060 Ti computeCapability: 8.9
coreClock: 2.655GHz coreCount: 34 deviceMemorySize: 15.71GiB deviceMemoryBandwidth: 268.25GiB/s
2023-11-28 21:33:00.219061: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2023-11-28 21:33:00.219070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2023-11-28 21:33:00.219075: I tensorflow/stream_executor/plat

## Save Predictions for Kyu
The same code block applies to Kyu by changing the model and dataset

In [6]:
# Load your own model. Here we use the baseline model
model = load_model('./model_bestkyu.h5')

# Load the corresponding dataset
df = open('./29_Public Testing Dataset_v2/kyu_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

## Save Predictions for PlayStyle
Playstyle requires only one predictions so the code here is a simple version of above

In [7]:
# Load your own model. Here we use the baseline model
model = load_model('./model_playstyle5.h5')

# Load the corresponding dataset
df = open('./29_Public Testing Dataset_v2/play_style_test_public.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')  
    moves_list = list(filter(lambda x: x != "", moves_list))
    x_testing.append(prepare_input_for_playstyle_model(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)

with open('./public_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)

2023-11-28 21:33:06.754788: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Now you can upload your file from the competition website! (https://tbrain.trendmicro.com.tw/Competitions/Details/29)

In [8]:
# Load your own model. Here we use the baseline model
model = load_model('./model_bestdan.h5')

# Load the corresponding dataset
df = open('./29_Private Testing Dataset_v2/dan_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

In [9]:
# Load your own model. Here we use the baseline model
model = load_model('./model_bestkyu.h5')

# Load the corresponding dataset
df = open('./29_Private Testing Dataset_v2/kyu_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')
    x_testing.append(prepare_input_for_dan_kyu_models(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_chars = top_5_preds_with_chars(predictions)

# Save results to public_submission_template.csv
with open('./public_submission_template.csv','a') as f:
    for index in range(len(prediction_chars)):
        answer_row = games_id[index] + ',' + ','.join(prediction_chars[index]) + '\n'
        f.write(answer_row)

In [10]:
# Load your own model. Here we use the baseline model
model = load_model('./model_playstyle5.h5')

# Load the corresponding dataset
df = open('./29_Private Testing Dataset_v2/play_style_test_private.csv').read().splitlines()
games_id = [i.split(',',2)[0] for i in df]
games = [i.split(',',2)[-1] for i in df]

x_testing = []

for game in games:
    moves_list = game.split(',')  
    moves_list = list(filter(lambda x: x != "", moves_list))
    x_testing.append(prepare_input_for_playstyle_model(moves_list))

x_testing = np.array(x_testing)
predictions = model.predict(x_testing)
prediction_numbers = np.argmax(predictions, axis=1)

with open('./public_submission_template.csv','a') as f:
    for index, number in enumerate(prediction_numbers):
        answer_row = games_id[index] + ',' + str(number+1) + '\n'
        f.write(answer_row)